# APIs and SQL Joins Lab

The city of San Francisco wants to assess the quality of restaurants in the city. Their data is scattered across multiple sources and incomplete.

They tasked you to help them assess it.

They would like to know what the most common violations are where they happen most frequently.

## 1. Initial data inspection

To answer the question we will need to retrieve and merge data from multiple files.

Yelp provides data on food quality, that can be found at [this address](http://www.yelp.com/healthscores/feeds). We already downloaded 4 files that you can find in the [assets folder](../../assets/datasets/yelp/).

In the bonus part we will also use the Google Geocoding API and data on [Neighborhoods](https://www.google.com/fusiontables/DataSource?docid=1zNwsvTwj-dH0QxuuDrKFsyfNklajd7WwEyaZ2U9M#rows:id=1).

1. Open each of the files and inspect them visually
- What information do they contain?

In [129]:
!ls ../../assets/datasets/yelp

businesses.csv  inspections.csv legend.csv      violations.csv


## 2. Local database

The first step in our analysis is to import the data into a local PostgreSQL database.

1. Launch a local Postgres server;
2. Create a local database named 'inspections';
3. Connect to it with sqlalchemy (see starter code) and import the files to separate tables.

**Note:** The files are not encoded in utf8, but in latin1. You can read more about encodings here: http://www.postgresql.org/docs/current/interactive/multibyte.html

In [130]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sqlalchemy import create_engine

engine = create_engine('postgresql://thomas@localhost:5432/inspections') # Replace 'winston' with your username

In [131]:
!ls ../../assets/datasets/yelp

businesses.csv  inspections.csv legend.csv      violations.csv


In [132]:
df1= pd.read_csv('../../assets/datasets/yelp/businesses.csv')
df2= pd.read_csv('../../assets/datasets/yelp/inspections.csv')
df3= pd.read_csv('../../assets/datasets/yelp/legend.csv')
df4= pd.read_csv('../../assets/datasets/yelp/violations.csv')
df1.to_sql(businesses,)

NameError: name 'businesses' is not defined

In [ ]:
def load_to_sql(name):
    df = pd.read_csv('../../assets/datasets/yelp/{}.csv'.format(name), encoding='latin1')
    df.to_sql(name, engine, if_exists='replace')
    print "done", name
    
load_to_sql('businesses')
load_to_sql('inspections')
load_to_sql('legend')
load_to_sql('violations')

### 2.b Display the first few lines of each table

In [ ]:
# Replace <user> with your user name

%sql postgresql://thomas@localhost:5432/inspections

In [ ]:
%%sql
select * from businesses LIMIT 5;

## 2.b Investigate violations

Let's focus on the violations table initially.


Answer these questions using sql:
1. How many violations are there?
- How many businesses committing violations?
- What's the average number of violations per business?

Answer these questions using python
1. Draw a plot of the violations count
- Is the average number of violations meaningful?
- Draw a plot of the normalized cumulative violation counts. Can we discard the restaurants with few violations?
- Where would you draw a threshold if you were to keep 90% of the violations?

In [ ]:
total=%sql SELECT count("business_id") FROM violations
total

In [ ]:
%sql SELECT * from violations limit 5

In [ ]:
counter= %sql SELECT COUNT(DISTINCT "business_id") FROM violations 
counter

In [ ]:
%sql SELECT CAST(count("business_id")/COUNT(DISTINCT "business_id") AS NUMERIC(5,2)) "AVG Violations per Business" from violations

In [ ]:
df= % sql SELECT * FROM violations
df=pd.DataFrame(df,columns=['index','business_id','date','description'])
temp=df.groupby("business_id")["description"].count()
plt.hist(temp)

### 2.c Investigate Inspections

In the previous step we looked at violations count. However we also have an inspection score available in the inspections table. Let's have a look at that too.

Answer these questions using SQL:
1. What's the average score for the whole city?
1. What's the average score per business?
- Does the score correlate with the number of inspections?
- Create a dataframe from a table with the following columns:
    business_id, average_score, number_of_inspections, number_of_violations
- Use pandas to do a scatter matrix plot of average_score, number_of_inspections, number_of_violations to check for correlations

In [ ]:
%sql select * from inspections limit 1

In [ ]:
%sql select AVG("score") from inspections

In [137]:
%sql select AVG("score") from inspections group by "business_id" limit 1

1 rows affected.


avg
94.0000000000000000


In [138]:
%sql select AVG("score"),count("score") from inspections group by "business_id" limit 3

3 rows affected.


avg,count
94.0000000000000000,2
59.0000000000000000,1
94.0000000000000000,1


In [155]:
%sql select inspections.business_id,max(inspections.date),inspections.score from inspections limit 5

(psycopg2.ProgrammingError) column "inspections.business_id" must appear in the GROUP BY clause or be used in an aggregate function
LINE 1: select inspections.business_id,max(inspections.date),inspect...
               ^
 [SQL: 'select inspections.business_id,max(inspections.date),inspections.score from inspections limit 5']


In [153]:
%%sql select inspections.business_id,max(inspections.date),inspections.score from violations inner join inspections on 
violations.business_id=inspections.business_id group by inspections.business_id limit 5;

(psycopg2.ProgrammingError) column "inspections.score" must appear in the GROUP BY clause or be used in an aggregate function
LINE 1: ...ect inspections.business_id,max(inspections.date),inspection...
                                                             ^
 [SQL: 'select inspections.business_id,max(inspections.date),inspections.score from violations inner join inspections on \nviolations.business_id=inspections.business_id group by inspections.business_id limit 5;']


In [143]:
df=%sql select * from violations v inner join inspections i on v."business_id"=i."business_id"

128926 rows affected.


In [198]:
data=pd.DataFrame(df,columns=["index","business_id","date","description","index_1","business_id_1",
                            "score","date_1","type"])
del data["index"]
del data["index_1"]
del data["business_id_1"]
del data["date_1"]
data=data[data["score"]!=np.NaN]
temp=data.groupby("business_id")["date"].count().to_frame()
temp=temp.reset_index()
data=data.merge(temp,on="business_id")

In [199]:
data["count"]=data["date_y"].copy()
del data["date_y"]
data["date"]=data["date_x"].copy()
del data["date_x"]

In [200]:
data

,business_id,description,score,type,count,date
0,10,Insufficient hot water or running water [ dat...,92,routine,10,20140729
1,10,Insufficient hot water or running water [ dat...,94,routine,10,20140729
2,10,Unapproved or unmaintained equipment or utensi...,92,routine,10,20140729
3,10,Unapproved or unmaintained equipment or utensi...,94,routine,10,20140729
4,10,Inadequate and inaccessible handwashing facili...,92,routine,10,20140114
5,10,Inadequate and inaccessible handwashing facili...,94,routine,10,20140114
6,10,Unclean or degraded floors walls or ceilings ...,92,routine,10,20140114
7,10,Unclean or degraded floors walls or ceilings ...,94,routine,10,20140114
8,10,Improper storage of equipment utensils or line...,92,routine,10,20140114
9,10,Improper storage of equipment utensils or line...,94,routine,10,20140114


In [187]:
data.corr()

,business_id,date_x,score,date_y
business_id,1.000000,0.058049,-0.059325,-0.019898
date_x,0.058049,1.000000,0.008211,-0.000048
score,-0.059325,0.008211,1.000000,-0.397523
date_y,-0.019898,-0.000048,-0.397523,1.000000


## 3 Zipcode analysis

The town administration would like to know which zip code are the ones where they should focus the inspections.

Use the information contained in the `businesses` table as well as the previous tables to answer the following questions using SQL:

1. Count the number of businesses per zipcode and sort them by descending order
- Which are the top 5 zipcodes with the worst average score?
    - restrict your analysis to the zipcodes with at least 50 businesses
    - do a simple average of the inspections scores in the postal code
- Which are the top 5 zipcodes with the highest number of violations per restaurant?
    - restrict your  analysis to the zipcodes with at least 50 businesses


## Final recommendation
Give a final recommendation on which 2 zipcodes should the administration focus and choose an appropriate plot to convince them visually.

## Bonus: Neighborhood data

Instead of looking at zipcodes we may be interested in using Neighborhood names.

It's beyond the scope of this lab to do a proper introduction to Geocoding and Reverse Geocoding, but we will give some pointers for further exploration.

### 1. Google Geocoding API
Have a look at:
- https://developers.google.com/maps/documentation/geocoding/intro
- https://maps.googleapis.com/maps/api/geocode/json?address=
- https://maps.googleapis.com/maps/api/geocode/json?latlng=

Through this API you can retrieve an address or a neighborhood from a lat-lon pair (reverse geocoding), or you can retrieve lat long and other information from an address (geocoding).

1. Try experimenting with and retrieving a few addresses
- Note that google imposes limits on the number of free queries
- How many missing lat-lon pairs do we have?

### Bonus 2
The pycurl library seems to be faster than requests in getting information from the google api.

1. See if you can extract the neighborhood from an address using the geocode api and a bit of json parsing
- Note that you would surely hit the daily limit if you pulled each address' neighborhood from the api

### Bonus 3
We can find the neighborhood using the polygons associated to each of them.
[Here](https://www.google.com/fusiontables/DataSource?docid=1zNwsvTwj-dH0QxuuDrKFsyfNklajd7WwEyaZ2U9M#rows:id=1) you can find these polygons (and we also copied them [locally](../../assets/datasets/sfneighborhoods.csv).

[This article](http://streamhacker.com/2010/03/23/python-point-in-polygon-shapely/) describes how to use the shapely package to check if a point belongs to a polygon.

- See if you can build a function that retrieves the neighborhood for a given address using the polygon data

### Further exploration

Postgres is actually GIS enabled, so we could do location based queries directly in the database.

Have a look at http://postgis.refractions.net/ for more information.